# Базовые операции с Kafka KRaft

## Создать топик

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic kraft-demo-topic \
    --create \
    --partitions 2 \
    --replication-factor 3

## Получить список топиков

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --list

## Записать данные в топик

In [ ]:
kafka-console-producer --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic kraft-demo-topic <<<$(yes 'Hello, World!' 2>/dev/null | head -n 10)

## Прочитать данные из топика

In [ ]:
kafka-console-consumer --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic kraft-demo-topic \
    --from-beginning \
    --timeout-ms 10000

## Утилиты для работы с Kraft

### Работа с идентификатором кластера

Перед стартом кластера необходимо указать `CLUSTER_ID`, значение для которго можно получить при помощи [`kafka-storage`](https://docs.confluent.io/kafka/operations-tools/kafka-tools.html#kafka-storage-sh)

In [ ]:
kafka-storage random-uuid

Посмотреть текущий идентификатор кластера можно при помощи следующей команды:

In [ ]:
kafka-storage info -c /etc/kafka/kraft/server.properties

### Получить статистику по рантайму

Текущее состояние кластера можно получить при помощи [kafka-metadata-quorum](https://docs.confluent.io/kafka/operations-tools/kafka-tools.html#kafka-metadata-quorum-sh). Например, следующая команда покажет общую информацию по кворуму:

In [ ]:
kafka-metadata-quorum --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    describe --status

### Получить содержимое файлов с логами

Утилита [kafka-dump-log](https://docs.confluent.io/kafka/operations-tools/kafka-tools.html#kafka-dump-log-sh) применяется для анализа содержимого файлов с логами Apache Kafka кластера. Логи в Apache Kafka хранят записанные в кластер события/записи.

Файлы с логами хранятся в `/tmp/kraft-combined-logs`

In [ ]:
HOST=kafka1 execute \
ls /tmp/kraft-combined-logs

#### Прочитать события из файла логов для топика `kraft-demo-topic`:

In [ ]:
log_file=$(HOST=${KAFKA_HOST} execute ls /tmp/kraft-combined-logs/kraft-demo-topic-0 | grep log)

HOST=kafka1 \
kafka-dump-log --print-data-log \
    --files /tmp/kraft-combined-logs/kraft-demo-topic-0/$log_file \
    --value-decoder-class kafka.serializer.StringDecoder

#### Прочитать события из топика метаданных

In [ ]:
log_file=$(HOST=${KAFKA_HOST} execute ls /tmp/kraft-combined-logs/__cluster_metadata-0 | grep log)

kafka-dump-log --cluster-metadata-decoder \
    --files /tmp/kraft-combined-logs/__cluster_metadata-0/${log_file} |
    head -n 15 |
    sed -n '/^\(baseOffset\||\)/s,\s,\n    ,gp' |
    sed -n '/^|/{d};/baseOffset/s,^,>>>>,;N;s,\n, ,p'

### Анализ лога с метаданными

Утилита [kafka-metadata-shell](https://docs.confluent.io/kafka/operations-tools/kafka-tools.html#kafka-metadata-shell.sh) позволяет в интерактивном режиме анализировать метаданные кластера. Утилита аналогична утилите [zookeeper-shell](https://docs.confluent.io/kafka/operations-tools/kafka-tools.html#zookeeper-shell-sh) при работе кластера Apache Kafka через Apache Zookeeper.

In [ ]:
log_file=$(HOST=${KAFKA_HOST} execute ls /tmp/kraft-combined-logs/__cluster_metadata-0 | grep log)

kafka-metadata-shell --snapshot /tmp/kraft-combined-logs/__cluster_metadata-0/${log_file} \
    ls / \
    2> /dev/null

In [ ]:
log_file=$(HOST=${KAFKA_HOST} execute ls /tmp/kraft-combined-logs/__cluster_metadata-0 | grep log)

kafka-metadata-shell --snapshot /tmp/kraft-combined-logs/__cluster_metadata-0/${log_file} \
    ls /brokers \
    2> /dev/null |
  sed 's, ,,g;s/\((\|,\)/\1\n/g'

In [ ]:
log_file=$(HOST=${KAFKA_HOST} execute ls /tmp/kraft-combined-logs/__cluster_metadata-0 | grep log)

kafka-metadata-shell --snapshot /tmp/kraft-combined-logs/__cluster_metadata-0/${log_file} \
    cat /brokers/201/registration \
    2> /dev/null |
  sed 's, ,,g;s/\((\|,\)/\1\n/g'

In [ ]:
log_file=$(HOST=${KAFKA_HOST} execute ls /tmp/kraft-combined-logs/__cluster_metadata-0 | grep log)

kafka-metadata-shell --snapshot /tmp/kraft-combined-logs/__cluster_metadata-0/${log_file} \
    cat /topics/kraft-demo-topic/0/data \
    2>/dev/null

### Получить информацию по кворуму

Утилита `kafka-metadata-quorum` позволяет получить метаданные для анализа Kafka кластера на базе KRaft.

Получить общую информацию о кластере:

In [ ]:
kafka-metadata-quorum --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    describe --status

Получить детали репликации журнала событий:

In [ ]:
kafka-metadata-quorum --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    describe --replication

## Удалить топик

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic kraft-demo-topic \
    --delete

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --list

## Задание

1. Увеличить количество контроллеров до 5
1. Убедитесь, что количество контроллеров равно 5
1. Какой контроллер является лидером?
1. Остановите контейнер лидера (`docker compose pause <LEADER SERVICE NAME>`)
1. Восстановите остановленный контейнер (`docker compose unpause <LEADER SERVICE NAME>`)
1. Увеличить количество брокеров до 7. Убедитесь, что количество брокеров равно 7
1. Почему необходимо использовать нечетные количество брокеров и контроллеров?